# ASP Rules

### Neural Atom

In [ ]:
 nn(label(1,I,B),["Aeroplane", "Animal_Wing", "Arm", "Artifact_Wing", "Beak", "Bicycle", "Bird",
                  "Boat", "Body", "Bodywork", "Bottle", "Bus", "Cap", "Car", "Cat", "Chain_wheel", "Chair",
                  "Coach", "Cow", "Dog", "Door", "Ear", "Ebrow", "Engine", "Eye", "Foot", "Hair", "Hand",
                  "Handlebar", "Head", "Headlight", "Hoof", "Horn", "Horse", "Leg", "License_plate",
                  "Locomotive", "Mirror", "Motorbike", "Mouth", "Muzzle", "Neck", "Nose", "Person",
                  "Plant", "Pot", "Pottedplant", "Saddle", "Screen", "Sheep", "Sofa", "Stern", "Tail",
                  "Torso", "Train", "Tvmonitor", "Wheel", "Window", "Diningtable","Other"]) :- box(I,B,X1,Y1,X2,Y2,C).


## Spatial Rules

In [ ]:
% Define the area of the bounding boxes
    area(B, A) :- box(_,B, Xmin,Ymin, Xmax, Ymax,_),
                A = (Xmax - Xmin) * (Ymax - Ymin).

    %Find if two bounding boxes overlap
    overlap(B1,B2):-box(_,B1,Xmin1,Ymin1,Xmax1,Ymax1,_),
                    box(_,B2,Xmin2,Ymin2,_,_,_),
                    B1!=B2,
                    Xmin2>=Xmin1,
                    Xmin2<=Xmax1,
                    Ymin2>=Ymin1,
                    Ymin2<=Ymax1.

    overlap(B1,B2):-box(_,B1,Xmin1,Ymin1,Xmax1,Ymax1,_),
                    box(_,B2,Xmin2,_,_,Ymax2,_),
                    B1!=B2,
                    Xmin2>=Xmin1,
                    Xmin2<=Xmax1,
                    Ymax2>=Ymin1,
                    Ymax2<=Ymax1.

    overlap(B1,B2):-box(_,B1,Xmin1,Ymin1,Xmax1,Ymax1,_),
                    box(_,B2,Xmin2,Ymin2,Xmax2,Ymax2,_),
                    B1!=B2,
                    Xmin1<Xmax2,
                    Xmin2<Xmax1,
                    Ymin1<Ymax2,
                    Ymin2<Ymax1.

    % Define the over90 predicate
    %Find if two bounding boxes overlap with over 90% coverage
    candidatePartOf(Bmin,Bmax) :- box(_,B1,Xmin1,Ymin1,Xmax1,Ymax1,_),
                    box(_,B2,Xmin2,Ymin2,Xmax2,Ymax2,_),
                    overlap(B1,B2),
                    area(B1, A1), area(B2, A2),
                    Amin='''+hashtag+r'''min{A1;A2},
                    Amax='''+hashtag+r'''max{A1;A2},
                    area(Bmin,Amin),
                    area(Bmax,Amax),
                    Ymax='''+hashtag+r'''min{Ymax1;Ymax2},
                    Ymin='''+hashtag+r'''max{Ymin1;Ymin2},
                    Xmax='''+hashtag+r'''min{Xmax1;Xmax2},
                    Xmin='''+hashtag+r'''max{Xmin1;Xmin2},
                    Aovl=(Ymax - Ymin) * (Xmax- Xmin),
                    90 <= ((100*Aovl)/ Amin).



##Knowledge Enrichment Rules

In [ ]:
    %partOf/3 inverse of hasTo/3
partOf(L2,L1,C):-hasA(L1,L2,C).

%partOf/3 inverse of hasContext/3
partOf(X,Y,C):-hasContext(X,Y,C).

% Transitivity

partOf(L1,L3,C):-
	partOf(L1,L2,C),
	partOf(L2,L3,C).


%Symmetricity
isSynonymWith(X,Y,C):-isSynonymWith(Y,X,C).

%Inference rules

partOf(X,Y,C):-
	partOf(X,Z,C),
	isA(Y,Z,C).

partOf(Z, Y,C) :-
    partOf(X, Y,C),
    isSynonymWith(X,Z,C).	

 partOf(X, Z,C) :-
     partOf(X, Y,C),
     isSynonymWith(Y,Z,C).


## Part Of Detection Rules

### PReDeCK  

In [ ]:

    %define boxes that contain objects
    objectBox(B,L,C):-label(_,_,B,L),object(L,C).

    %define boxes that contain parts
    partBox(B,L,C):-label(_,_,B,L),part(L,C).


    %define boxes that have a Spatial partOf relation
    isPartOf(B1,L1,B2,L2):-candidatePartOf(B1,B2),
                    partBox(B1,L1,C),
                    objectBox(B2,L2,C),
                    partOf(L1,L2,C).



### Baseline method

Uses only the candidatePartOf/2 to detect Part of relations

###PReDeCK<sub>GT1</sub>    

Considers only the type and category of the labels.

In [ ]:
 isPartOf(B1,L1,B2,L2):-candidatePartOf(B1,B2),
                    partBox(B1,L1,C),
                    objectBox(B2,L2,C).


###PReDeCK<sub>GT2</sub>

Uses the same rules as PReDeCK, but as a knowledge domain, it has the information derived from the ground truth about all the possible partOf relations present in the dataset.

###PReDeCK<sub>CN1<sub>

Uses the same rules as PReDeCK, but it's knowledge domain only includes the labels’ types and categories along with information that is
directly accessible through the edge “PartOf” of Concept-
Net.

###PReDeCK<sub>CN2<sub>

Doesn't consider the type and categories of the labels

In [ ]:
isPartOf(B1,L1,B2,L2):-candidatePartOf(B1,B2),
                    label(_,_,B1,L1),
                    label(_,_,B2,L2),
                    partOf(L1,L2,_).


### Error Detection

In [ ]:
%%PRELIMINARIES

        highConfidence(B):-box(_,B,_,_,_,_,C),
                        C >= 400.

        hasNumberOfAccParts(Y,N):-N='''+hashtag+r'''count {(X,Y):semantic_partOf(X,_,Y,_),partBox(X,_,C,_),C>=400},objectBox(Y,_,_,_).

        context(B):-hasNumberOfAccParts(B,N), N >= 2.

        single(B):-partBox(B,_,_,_), not semantic_partOf(B,_,_,_).

        knownPart(B):-partBox(B,L,_,_),partOf(L,_,_).
        knownObject(B):-objectBox(B,L,_,_),partOf(_,L,_).

        %%RATIONALE
        %%Error Case P on the publication
        errorCaseA(B1):-spatial_partOf(B1,_,B2,_),
                    knownPart(B1),
                    knownObject(B2),
                    highConfidence(B2),
                    not highConfidence(B1),
                    single(B1).
        %%Error Case O on the publication
        errorCaseB(B2):-spatial_partOf(B1,_,B2,_),
                    knownPart(B1),
                    knownObject(B2),
                    highConfidence(B1),
                    not highConfidence(B2),
                    not context(B2),
                    single(B1).
        %%Error Case PO on the publication
        errorCaseAB(B1,B2):-spatial_partOf(B1,_,B2,_),
            knownPart(B1),
            knownObject(B2),
            not highConfidence(B1),
            not highConfidence(B2),
            not context(B2),
            single(B1).
        %%Error Case MO on the publication
        errorCaseC(B1):-spatial_partOf(B1,_,B2,_),
            knownPart(B1),
            knownObject(B2),
            highConfidence(B1),
            1{highConfidence(B2);context(B2)},
            single(B1).

        errorCaseC(B1):-single(B1),
                        knownPart(B1),
                        highConfidence(B1),
                        not spatial_partOf(B1,_,_,_).
        

### Error Handling

## Wrong Coordinates Filtering Rule
Set inclusion ratio vary low.

In [ ]:
over90(Bmin,Bmax) :- box(_,B1,Xmin1,Ymin1,Xmax1,Ymax1,_), 
                     box(_,B2,Xmin2,Ymin2,Xmax2,Ymax2,_),
                     overlap(B1,B2), area(B1, A1), area(B2, A2), 
                     Amin=+'''hashtag'''+min{A1;A2}, Amax=+'''hashtag'''+max{A1;A2}, area(Bmin,Amin), 
                     area(Bmax,Amax), Ymax=+'''hashtag'''+min{Ymax1;Ymax2}, Ymin=+'''hashtag'''+max{Ymin1;Ymin2},
                     Xmax=+'''hashtag'''+min{Xmax1;Xmax2}, Xmin=+'''hashtag'''+max{Xmin1;Xmin2}, 
                     Aovl=(Ymax - Ymin) * (Xmax- Xmin), 5 <= ((100*Aovl)/ Amin).')


## MO Error handling rules

In [ ]:
##For ConceptNet as Knowledge Domain
    ## Find possible missing objects
    isNotUniquePart(X,C):-partOf(X,Y,C),partOf(X,Z,C),Y!=Z.

    isUniquePartOf(X,Y,C):-part(X,C),object(Y,C),partOf(X,Y,C),not isNotUniquePart(X,C).

    possibleClass(B,Y,C):-single(B),errorCaseC(B),partBox(B,L,_,C),object(Y,C),partOf(L,Y,C).

    detClass(Y,C):-single(B),partBox(B,L,_,C),object(Y,C),errorCaseC(B),isUniquePartOf(L,Y,C).
    ##Quantify missing objects
    in_category(Y,Cat):-possibleClass(_,Y,Cat).

    numMissingobjects(N):- N='''+hashtag+r'''count{Y: in_category(_,Y)}.
##For Ontology as Knowledge Domain
    ## Find possible missing objects
    isNotUniquePart(X):-partOf(X,Y),partOf(X,Z),Y!=Z.

    isUniquePartOf(X,Y):-part(X),object(Y),partOf(X,Y),not isNotUniquePart(X).

    possibleClass(B,Y):-single(B),errorCaseC(B),label(_,_,B,L),partOf(L,Y).

    detClass(Y):-single(B),label(_,_,B,L),errorCaseC(B),isUniquePartOf(L,Y).

    ##Quantify missing objects
    numPosClasses(N):-N='''+hashtag+r'''count{X:possibleClass(_,X)}.

    category(1..N):-numPosClasses(N).

    {in_category(S, C) : possibleClass(_,S), category(C)}.

    :-0{in_category(_,_)}N-1,numPosClasses(N).

    sharePart(Y1,Y2):-partOf(X,Y1),partOf(X,Y2),Y1!=Y2.

    :- in_category(S1, C), in_category(S2, C), S1 != S2, not sharePart(S1,S2).

    :- in_category(S1, C1), in_category(S2, C2), C1 != C2, partOf(P,S1), partOf(P,S2).

    numMissingobjects(N):- N='''+hashtag+r'''count{Y: in_category(_,Y)}.



